In [1]:
# FileNotFoundError: [WinError 2] The system cannot find the file specified
# https://www.geeksforgeeks.org/how-to-install-ffmpeg-on-windows/
import whisper
from pytube import YouTube
from transformers import pipeline
import os
import pandas as pd
from tqdm import tqdm

In [2]:
model = whisper.load_model("base")
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
df = pd.read_csv("../data/video_transcription.csv")
df.head(5)

,title,video_id,url,channel,parent_id,parent,depth,lang,transcript
0,China Navy to BBC: 'Stay away from islands' - ...,LVeKbEgn50o,https://www.youtube.com/watch?v=LVeKbEgn50o,BBC News,nbBsh46aSz4,https://www.youtube.com/watch?v=nbBsh46aSz4,1,en,it's just before dawn on the Philippine islan...
1,"Nike, H&M face backlash in China over Uighur s...",qC_qOoBFsMc,https://www.youtube.com/watch?v=qC_qOoBFsMc,DW News,s_eNkqwp_LA,https://www.youtube.com/watch?v=s_eNkqwp_LA,1,en,u.s border para maker nike is facing a backla...
2,What do you do when someone just doesn't like ...,Y4gly9n9RBo,https://www.youtube.com/watch?v=Y4gly9n9RBo,TEDx Talks,s_eNkqwp_LA,https://www.youtube.com/watch?v=s_eNkqwp_LA,1,en,Translator: Rhonda Jacobs Reviewer: Tanya Cush...
3,99 Years Later... We Solved It,JFFhD5HeByM,https://www.youtube.com/watch?v=JFFhD5HeByM,Physics Girl,tcKw5jiT6wg,https://www.youtube.com/watch?v=tcKw5jiT6wg,1,en,look at this okay so here's a trail and the t...
4,"Elon Musk talks Twitter, Tesla and how his bra...",cdZZpaB2kDM,https://www.youtube.com/watch?v=cdZZpaB2kDM,TED,tcKw5jiT6wg,https://www.youtube.com/watch?v=tcKw5jiT6wg,1,en,hello so in just a few minutes um elon musk ...


In [4]:
# Find the video_ids that has no transcription
no_captions = df[df['transcript'] == 'No transcription']['video_id'].unique()
no_captions[:5]

array(['L0oo8PsUKrA', '2Shla50pRxA', 'nbBsh46aSz4', 'tcKw5jiT6wg',
       'nfGkJAX2WUc'], dtype=object)

In [5]:
def get_audio(url):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    out_file=video.download(output_path="videos\\.")
    base, ext = os.path.splitext(out_file)
    new_file = base+'.mp3'
    os.rename(out_file, new_file)
    return new_file

def get_text(video_id):
    url = 'https://www.youtube.com/watch?v=' + video_id
    file = get_audio(url)
    result = model.transcribe(file)
    return result['text']

In [6]:
# Slice the data if you need.
no_cap_slice = no_captions[:]

In [17]:
transcriptions = []
for video_id in tqdm(no_cap_slice):
    try:
        transcription = get_text(video_id)
        transcriptions.append(transcription)
    except:
        # Live show, Not available, removed
        transcriptions.append("Not Video")

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [3:52:21<00:00, 278.83s/it]


In [19]:
# Save in csv file with video_id
pd.DataFrame({'video_id':no_cap_slice, 'transcription': transcriptions}).to_csv('video_transcription_with_whisper.csv', index=False)